In [2]:
import pandas as pd
import numpy as np
import os
import math


from pyspark.sql.session import SparkSession
# spark세션 인스턴스
spark = SparkSession.builder.appName('master').getOrCreate()

from pyspark.sql.types import *
from pyspark.sql.functions import desc, asc
from pyspark.sql import functions as f
from pyspark.sql.functions import col

In [3]:
from pywebhdfs.webhdfs import PyWebHdfsClient
from pprint import pprint
 

# path안에 있는 파일 및 폴더이름을 리스트로 반환한다
# 단 path는 'user/hadoop/data_analysis/coupang_data/' 와 같이 http를 기입하지 않는다
def get_dir(path):
    hdfs = PyWebHdfsClient(host='localhost',port='50070', user_name='master')  # your Namenode IP & username here
    my_dir = path
    
    result = []
    
    for i in range(len(hdfs.list_dir(my_dir)['FileStatuses']['FileStatus'])):
        result.append(hdfs.list_dir(my_dir)['FileStatuses']['FileStatus'][i]['pathSuffix'])
    
    return result

# pandas dataframe으로 csv파일을 열고 dataframe을 반환한다
# 단 path는 'user/' 로 시작
def open_pd_csv(path):
    file_path = 'hdfs://192.168.0.9:9000/' + path
    data = spark.read.csv(file_path, header=True)
    result = data.toPandas()
    
    return result

# pandas dataframe을 path에 저장한다
# 단 path는 'user/' 로 시작 , 파일명까지 포함 ex) uesr/hadoop/test.csv
def save_pd_csv(df ,path):
    file_path = 'hdfs://192.168.0.9:9000/' + path
    data = spark.createDataFrame(df)
    data.coalesce(1).write.mode("overwrite").option("header","true").csv(file_path)


    
def save_pd_parquet(df ,path):
    file_path = 'hdfs://192.168.0.9:9000/' + path
    data = spark.createDataFrame(df)
    data.write.mode("overwrite").format("parquet").save(file_path)

    
# spark datafrmae schema 설정
def ad_schema(df):
    intCols = ['rank','category','price', 
               'discount_percentage', 'rating_total_count',
               'reviews_for_last1year', 'sales']
    doubleCols = ['rating']
    boolCols = ['rocket_delivery', 'is_out_of_stock']
        
    for c in df.columns:
        type_col = StringType()
        if c in intCols:
            type_col = IntegerType()
        elif c in doubleCols:
            type_col = DoubleType()
        elif c in boolCols:
            type_col = BooleanType()
        
        df = df.withColumn(c,df[c].cast(type_col))
    df = df.fillna(0)
    return df

In [ ]:
# merge_csv
path_dir = 'user/hadoop/data_analysis/coupang_data/'
dir_list = get_dir(path_dir)

for d in dir_list:
    base = path_dir + d + '/'
    file_list = get_dir(base)
    #파일목록에서 py파일 삭제
    for f in file_list[:15]:
        if '.py' in f:
            file_list.remove(f)
    
    if 'getReviewData.py' in file_list:
        file_list.remove('getReviewData.py')

    
    # 상품정보 취합
    data = open_pd_csv(base + file_list[0])
    
    for file in file_list[1:9]:
        temp = open_pd_csv(base + file)
        data = pd.concat([data,temp])

        # 인덱스 리셋(1~120까지 반복되는 현상 때문에 작성)
        data = data.reset_index()
        data = data.drop(['index'],axis=1)
    
    # result.csv 목록에서 삭제
    del file_list[len(file_list)-1]
    
    # 리뷰정보 취합
    data2 = open_pd_csv(base + file_list[9])
    
    k = 10
    
    for file in file_list[10:]:
        if (file == 'result.csv') or (file == 'result_review.csv'):
            continue
        print(base, file, k)
        
        #temp = pd.read_csv(base + file, error_bad_lines=False)
        temp = open_pd_csv(base + file)
        data2 = pd.concat([data2,temp])
        
        temp = None
        
        # 인덱스 리셋(1~120까지 반복되는 현상 때문에 작성)
        data2 = data2.reset_index()
        data2 = data2.drop(['index'],axis=1)
        k+=1
    
        
    #result 데이터프레임에 상품별 리뷰정보 추가
    product_id = file[14:file.find('csv')-1]
    df_temp = data2[data2['reg_date'] > '2020.06.01'] # 2020년 6월 1일이후 리뷰만 필터
    df_temp = df_temp.groupby('product_id').agg(['count'])
    df_temp = df_temp.ratings
    data = pd.merge(left = data, right = df_temp, how = 'left', on = 'product_id')
    
    # 중복 제거
    data = data.drop_duplicates(['product_id'])

    # 상품별 상품정보 취합된 csv파일 저장
    save_pd_parquet(data ,base + 'transformed.parquet')
    
    # 상품별 상품정보 취합된 csv파일 저장
    save_pd_csv(data ,base + 'result.csv')
    data = None
    #result_review.csv에 정상적으로 저장이 안됨
    #save_pd_csv(data2 ,base + 'result_review.csv')
    data2 = None

user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_102407096.csv 10
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_102580500.csv 11
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_104390448.csv 12
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_1058415715.csv 13
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_1058913221.csv 14
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_1060193441.csv 15
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_10607696.csv 16
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_1062838712.csv 17
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_1063372010.csv 18
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_1065945065.csv 19
user/hadoop/data_analysis/coupang_data/bed (c

user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_128301263.csv 96
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_128659939.csv 97
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_1291494178.csv 98
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_1291494197.csv 99
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_129481818.csv 100
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_129944613.csv 101
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_130006662.csv 102
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_1301183620.csv 103
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_1306926674.csv 104
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_1310197751.csv 105
user/hadoop/data_analysis/coupang_data/

user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_1588938873.csv 181
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_1597728939.csv 182
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_1598404584.csv 183
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_160136871.csv 184
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_1604677955.csv 185
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_1606666689.csv 186
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_1610964169.csv 187
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_161265796.csv 188
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_161349239.csv 189
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_1614715960.csv 190
user/hadoop/data_analysis/coupang

user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_1826430249.csv 267
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_182789092.csv 268
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_1829493113.csv 269
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_1829493909.csv 270
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_1831465812.csv 271
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_1833845262.csv 272
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_18342378.csv 273
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_1837047118.csv 274
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_184631206.csv 275
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_184631209.csv 276
user/hadoop/data_analysis/coupang_d

user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_2082412064.csv 352
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_2083440667.csv 353
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_2086361746.csv 354
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_2086389955.csv 355
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_2087297107.csv 356
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_2087915123.csv 357
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_2091558146.csv 358
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_2093548879.csv 359
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_2104336801.csv 360
user/hadoop/data_analysis/coupang_data/bed (category = 184562)/ output_review_2112099563.csv 361
user/hadoop/data_analysis/coup

...

user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_2341046751.csv 424
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_234170079.csv 425
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_2353946276.csv 426
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_235559353.csv 427
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_236997139.csv 428
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_237076176.csv 429
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_237088176.csv 430
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_2371467905.csv 431
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_2371470154.csv 432
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_23755350.csv 433
user/hadoop/data_

user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_311700143.csv 510
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_316002787.csv 511
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_319683974.csv 512
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_319808928.csv 513
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_324650917.csv 514
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_327265591.csv 515
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_33340928.csv 516
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_334937108.csv 517
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_335065423.csv 518
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_335659855.csv 519
user/hadoop/data_anal

user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_4656244394.csv 594
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_4660692110.csv 595
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_4703447.csv 596
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_47059911.csv 597
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_4711881539.csv 598
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_4711881578.csv 599
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_4729832494.csv 600
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_4735553384.csv 601
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_4735553433.csv 602
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_4735554097.csv 603
user/hadoop/dat

user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_5233682564.csv 679
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_5237338348.csv 680
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_5237731531.csv 681
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_5239656959.csv 682
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_5253495070.csv 683
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_5253495253.csv 684
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_5256842286.csv 685
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_5263662343.csv 686
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_52695115.csv 687
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_5278809072.csv 688
user/hadoop/

user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_88514434.csv 764
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_88514461.csv 765
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_88514467.csv 766
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_8929398.csv 767
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_895935.csv 768
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_89672089.csv 769
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_89695338.csv 770
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_90006875.csv 771
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_9003684.csv 772
user/hadoop/data_analysis/coupang_data/chair (category = 184634)/ output_review_9049682.csv 773
user/hadoop/data_analysis/coupang_d

In [3]:
# split_inner_attribute
def innerTocol(df, attr):
    """ df : detail이라는 column을 가진 데이터프레임 (type : dataframe)
        attr : detail에서 찾아올 내적 특성, column명이 됨 (type : str)
        반환값은 없으며 함수 내부에서 df에 새로운 column이 추가됨 """
    
    # 추출한 내적특성의 값을 담을 list
    inner = []
    
    # 내적 특성 값 추출
    for s in df.attribute_list:
        if type(s) != str:
            inner.append('NULL')
        elif s.find(attr) != -1:
            #print(s[s.find(":",s.find(attr))+2:s.find("'",s.find(attr))])
            inner.append(s[s.find(":",s.find(attr))+2:s.find("'",s.find(attr))])
        else:
            inner.append('NULL')
    
    # 데이터프레임에 새로운 열 추가
    df[attr] = inner

# 상품별 내적특성 설정
attr_list = [['인테리어', '재질', '사이즈', '설치', '색상', '매트리스포함', '프레임'],
             ['사이즈', '색상', '형태', '인테리어', '재질', '접이'],
             ['사이즈', '색상', '소재', '재질', '팔걸이', '등받이', '종류', '인테리어'],
             ['사이즈', '색상', '재질', '인테리어'],
             ['사이즈', '색상', '재질', '인테리어'],
             ['사이즈', '색상', '재질', '인테리어', '스툴', '거울'],
             ['사이즈', '색상', '재질', '인테리어', '종류', '타입', '거울'],
             ['색상', '재질', '인테리어', '논슬립', '묶음', '종류', '회전'],
             ['용량', '개폐방식', '재질', '투명', '접이', '바퀴', '손잡이'],
             ['사이즈', '색상', '재질', '형태', '바퀴'],
             ['사이즈', '색상', '발통', '매트/토퍼', '두께', '겉감재질', '충전재'],
             ['색상', '재질', '접이', '바퀴', '사용인원'],
             ['색상', '재질', '인테리어'],
             ['사이즈', '색상', '재질', '소재', '인테리어', '방식', '각도조절'],
             ['사이즈', '잠금방식', '중량', '경보기', '지문인식'],
             ['단', '재질', '형태'],
             ['사이즈', '색상', '재질', '인테리어'],
             ['사이즈', '색상', '재질', '소재', '스툴', '사용인원', '설치', '인테리어'],
             ['사이즈', '색상', '재질', '인테리어', '바퀴', '단'],
             ['사이즈', '색상', '재질', '인테리어', '가로길이', '형태']]

item_list = ['bed', 'bedtray', 'chair', 'closet', 'drawers', 'dressingtable,console',
            'hanger', 'hanger,doorhook', 'livingbox', 'livingroom table', 'mattress',
            'outdoor furniture', 'partition', 'recliner', 'safe', 'shoes shelf', 'smalltable',
            'sofa', 'storage', 'table']

inner_attr = {}
for i in range(20):
    inner_attr[item_list[i]] = attr_list[i]
    
# 내적특성을 열로 추가
path_dir = 'user/hadoop/data_analysis/coupang_data/'
dir_list = get_dir(path_dir)

for d in dir_list:
    base = path_dir + d + '/'
    file_name = 'result.csv'
    item = d[:d.find('(')-1]
    data = open_pd_csv(base + file_name)
    attrs = inner_attr[item]
    for a in attrs:
        innerTocol(data, a)
    
    # 필요없는 열 삭제
    data = data.drop('attribute_list',axis=1)
    data = data.drop('baby_product_link',axis=1)
    data = data.drop('recommends_list',axis=1)
    
    # 색상 열에서 "계열"문자열 모두 삭제
    if '색상' in data.columns:
        data["색상"] = data["색상"].str.replace("계열","")
    data = data.rename(columns = {"count":"reviews_for_last1year"})
        
    save_pd_csv(data ,'user/hadoop/data_analysis/merged_data/'+ item + '_result.csv')

In [4]:
# 판매량 산정 후 결과 저장
basic_folder = 'user/hadoop/data_analysis/merged_data/'
file_list = get_dir(basic_folder)

for file in file_list:
    file_name = basic_folder + file

    # 스파크 dataframe 생성(csv파일 읽기)
    df = spark.read.csv('hdfs://192.168.0.9:9000/' + file_name, header=True)
    df.show()
    
    # 스키마 정의
    intCols = ['rank','category','price', 'discount_percentage', 'rating_total_count',
                  'reviews_for_last1year']
    doubleCols = ['rating']
    boolCols = ['rocket_delivery', 'is_out_of_stock']
        
    for c in df.columns:
        type_col = StringType()
        if c in intCols:
            type_col = IntegerType()
        elif c in doubleCols:
            type_col = DoubleType()
        elif c in boolCols:
            type_col = BooleanType()
        
        df = df.withColumn(c,df[c].cast(type_col))
    df = df.fillna(0)
    
    # 카테고리 출력
    category = file[:file.find('_')]
    print('카테고리 : ',category)

    # 판매량 산정
    temp = df.select('reviews_for_last1year').sort(asc('reviews_for_last1year'))
    temp = df.select('reviews_for_last1year').filter(df.reviews_for_last1year > 0).\
    sort(asc('reviews_for_last1year')).collect()[0:10]
    
    std = 0
    for r in temp :
        std += r['reviews_for_last1year']
    std = std/10

    df = df.withColumn('sales', (1080 - df.rank)*std + (df.reviews_for_last1year)*3)
    
    # 결과 저장(parquet로 저장)
    file_path = 'hdfs://192.168.0.9:9000/' + 'user/hadoop/data_analysis/result2/' + category + '.parquet'
    df.coalesce(1).write.mode("overwrite").option("header","true").format("parquet").save(file_path)

+----+--------+--------------------+------+----------+--------+-------------------+------+------------------+---------------+----------+------------+---------------------+------+----+--------------------+----+----+------+----+
|rank|category|                name| price|product_id|isRocket|discount_percentage|rating|rating_total_count|is_out_of_stock|brand_name|shopping_fee|reviews_for_last1year|  인테리어|  재질|                 사이즈|  설치|  색상|매트리스포함| 프레임|
+----+--------+--------------------+------+----------+--------+-------------------+------+------------------+---------------+----------+------------+---------------------+------+----+--------------------+----+----+------+----+
|   1|  184562|라꾸라꾸 수납침대 4탄 접이식침...|149800|  10755024|    true|                  0|   4.5|              1436|          False|      라꾸라꾸|           0|                402.0|  NULL|NULL|                  싱글|NULL|NULL|  NULL|NULL|
|   2|  184562|북유럽 소나무 침대 받침대 프레...| 39800|4899755280|   false|                 33|   4.5|  

In [5]:
# 카테고리별 인기 요소 추출
basic_folder = 'user/hadoop/data_analysis/result2/'
file_list = get_dir(basic_folder)

for file in file_list:
    
    if '.csv' in file:
        continue
    
    # 해당하는 카테고리의 데이터프레임 가져오기
    file_name = basic_folder + file
    # df = spark.read.csv('hdfs://192.168.0.9:9000/' + file_name, header=True)
    df = spark.read.parquet('hdfs://192.168.0.9:9000/' + file_name)
    df = ad_schema(df)
    
    #카테고리 출력
    item = file[:file.find('.')]
    print('카테고리 : ',item)

    # 상품별 인기 요소
    keywords = []
    
    # 특성별 판매량 순위
    for col in df.columns[13:]:
        #print('특성 :', col)
        result = df.groupby(col).agg(f.count(col).alias("개수"),
                                     f.sum('sales').alias('판매량')).sort(desc('판매량'))
        result = result.withColumn('상품당 판매량',(result.판매량/result.개수)).sort(desc('상품당 판매량'))
        result = result.filter((f.col('개수') > 9) & (f.col(col) != 'NULL'))
        
        if result.count() != 0:
            key = str(result.select(col).take(1)[0])
            key = key[key.find("=")+2:key.find(")")-1]
            keywords.append(col+ ' : ' +key)
        
        #result.show()
    print("인기 요소 :", keywords)
    print('-------------------------------------------------------------------------------------------------------------------')

카테고리 :  bed
인기 요소 : ['인테리어 : 모던/심플', '재질 : 합성목재', '사이즈 : 싱글', '설치 : 방문설치', '색상 : 그레이', '매트리스포함 : 매트리스포함', '프레임 : 밀폐형']
-------------------------------------------------------------------------------------------------------------------
카테고리 :  bedtray
인기 요소 : ['색상 : 브라운', '형태 : 직사각형', '인테리어 : 모던/심플', '재질 : 원목', '접이 : 접이식가능']
-------------------------------------------------------------------------------------------------------------------
카테고리 :  chair
인기 요소 : ['색상 : 베이지', '소재 : 플라스틱', '재질 : 철제', '팔걸이 : 팔걸이없음', '등받이 : 등받이없음', '종류 : 접이식의자', '인테리어 : 모던/심플']
-------------------------------------------------------------------------------------------------------------------
카테고리 :  closet
인기 요소 : ['색상 : 그레이', '재질 : PB + MDF', '인테리어 : 내추럴']
-------------------------------------------------------------------------------------------------------------------
카테고리 :  drawers
인기 요소 : ['색상 : 화이트', '재질 : 철제', '인테리어 : 내추럴']
------------------------------------------------------------------------------